In [1]:
import requests
import random
import json

base_url = "http://localhost:8081"

resp = requests.get(base_url)
assert resp.status_code == 404
assert resp.json()["status"] == 404
assert resp.json()["error"] == "Not Found"


In [2]:

resp = requests.get("%s/api/docs/swagger.json"%base_url)
assert resp.status_code == 200
assert resp.json()["swagger"] == "2.0"
assert resp.json()["paths"] != None

swaggerJson = resp.json()
swaggerJson["paths"].keys()

jsonUrl = [f for f in swaggerJson["paths"].keys() if ".json" in f][0]
jsonUrl

resp = requests.get("%s%s"%(base_url, jsonUrl))
assert resp.status_code == 200


In [3]:
resp.json()

{'TrainingCertificate': {'$id': '#/properties/TrainingCertificate',
  'type': 'object',
  'title': 'The TrainingCertificate Schema',
  'required': ['name', 'contact'],
  'properties': {'name': {'type': 'string'},
   'trainingTitle': {'type': 'string'},
   'contact': {'type': 'string'},
   'date': {'type': 'string', 'format': 'date'},
   'note': {'type': 'string'}}}}

In [4]:
entities = list(resp.json().keys())
print("Available entities ", entities)

Available entities  ['TrainingCertificate']


In [5]:
entity_name=entities[0]
print("Using entity %s"%entity_name)


Using entity TrainingCertificate


In [6]:
userId =str(random.randint(1e10,1e11))
resp = requests.post("%s%s"%(base_url, '/api/v1/%s'%entity_name), json={
    "name":"Sunbird Learner", 
    "contact": userId, 
    "trainingTitle":"Sunbird RC Certificate Module"
    
})
assert resp.status_code == 200 or print (resp.json())
idx = resp.json()["result"][entity_name]["osid"]


In [7]:
resp.json()


{'id': 'sunbird-rc.registry.create',
 'ver': '1.0',
 'ets': 1645117010600,
 'params': {'resmsgid': '',
  'msgid': '7c5472ef-fd75-4b8f-8877-3cb2e0e31077',
  'err': '',
  'status': 'SUCCESSFUL',
  'errmsg': ''},
 'responseCode': 'OK',
 'result': {'TrainingCertificate': {'osid': '1-8589ca85-ab48-4bcd-ad7c-9548553a0995'}}}

In [8]:
print(resp.json())
resp.status_code, userId


{'id': 'sunbird-rc.registry.create', 'ver': '1.0', 'ets': 1645117010600, 'params': {'resmsgid': '', 'msgid': '7c5472ef-fd75-4b8f-8877-3cb2e0e31077', 'err': '', 'status': 'SUCCESSFUL', 'errmsg': ''}, 'responseCode': 'OK', 'result': {'TrainingCertificate': {'osid': '1-8589ca85-ab48-4bcd-ad7c-9548553a0995'}}}


(200, '85764892031')

In [9]:
resp = requests.get("%s/api/v1/%s/%s"%(base_url, entity_name, idx), headers={"Accept":"application/vc+ld+json"})
print(json.dumps(resp.json()))
assert resp.json()["proof"]["type"] == "Ed25519Signature2018"


{"@context": ["https://www.w3.org/2018/credentials/v1", "https://gist.githubusercontent.com/dileepbapat/eb932596a70f75016411cc871113a789/raw/498e5af1d94784f114b32c1ab827f951a8a24def/skill"], "type": ["VerifiableCredential"], "issuanceDate": "2021-08-27T10:57:57.237Z", "credentialSubject": {"type": "Person", "name": "Sunbird Learner", "trainedOn": "Sunbird RC Certificate Module"}, "issuer": "did:web:sunbirdrc.dev/vc/skill", "proof": {"type": "Ed25519Signature2018", "created": "2022-02-17T16:56:51Z", "verificationMethod": "did:india", "proofPurpose": "assertionMethod", "jws": "eyJhbGciOiJFZERTQSIsImI2NCI6ZmFsc2UsImNyaXQiOlsiYjY0Il19..NJOK8YrkkucvYFwcf_P6-0CIg3Lbhu5GnAtPwbjkuSLRlhwFKGk8oHkZszs7RxwCcz1DG-hCIispLvDx_0aSCA"}}


In [10]:
resp = requests.get("%s/api/v1/%s/%s"%(base_url, entity_name, idx), headers={"Accept":"application/pdf"})


In [11]:
resp.status_code, resp.content

assert resp.content[:5].decode().startswith("%PDF")
with open('sample.pdf', 'wb') as f:
    f.write(resp.content)
    

In [12]:
%system open 'sample.pdf'


[]

In [13]:
resp = requests.get("%s/api/v1/%s/%s"%(base_url, entity_name, idx), headers={"Accept":"text/html"})
resp.status_code, resp.content

assert resp.content[:5].decode().startswith("<html")
with open('sample.html', 'wb') as f:
    f.write(resp.content)
%system open 'sample.html'


[]

In [14]:
resp = requests.get("%s/api/v1/%s/%s"%(base_url, entity_name, idx), headers={"Accept":"image/svg+xml"})
resp.status_code, resp.content

assert resp.content.decode().find("<svg") != -1
with open('sample.svg', 'wb') as f:
    f.write(resp.content)
%system open 'sample.svg'

[]